In [1]:
import pandas as pd
import numpy as np

In [2]:
mainData = pd.read_csv("vgscore.csv")
mainData = mainData.drop(labels = ['url', 'tot_ship', 'tot_sale', 'na_sale', 'pal_sale', 'jp_sale', 'other_sale', 'last_update'], axis = 1)
mainData.rename(columns = {'title' : 'Name'}, inplace = True)
mainData

,Name,user_Score,release_Date,console,publisher,developer,VGscore
0,Wii Sports,N/A,19th Nov 06,Wii,Nintendo,Nintendo EAD,N/A
1,Super Mario Bros.,8.2,18th Oct 85,NES,Nintendo,Nintendo EAD,N/A
2,Mario Kart Wii,9.1,27th Apr 08,Wii,Nintendo,Nintendo EAD,8.7
3,PlayerUnknown's Battlegrounds,N/A,21st Dec 17,PC,PUBG Corporation,PUBG Corporation,N/A
4,Wii Sports Resort,8.8,26th Jul 09,Wii,Nintendo,Nintendo EAD,8.8
...,...,...,...,...,...,...,...
55859,ZOMBIE GOLD RUSH,N/A,26th Oct 17,NS,Amazing Inc.,Amazing Inc.,N/A
55860,Zombillie,N/A,29th Mar 18,NS,Forever Entertainment S.A.,Forever Entertainment S.A.,N/A
55861,Zone of the Enders: The 2nd Runner MARS,N/A,06th Sep 18,PC,Konami,Cygames,N/A
55862,Zoo Tycoon: Ultimate Animal Collection,N/A,31st Oct 17,XOne,Microsoft Studios,Frontier Developments,N/A


In [3]:
#sorry this is slow but the data has every name entry having spaces...
for c in range(len(mainData)):
    mainData["Name"][c] = mainData["Name"][c].strip()
mainData

,Name,user_Score,release_Date,console,publisher,developer,VGscore
0,Wii Sports,N/A,19th Nov 06,Wii,Nintendo,Nintendo EAD,N/A
1,Super Mario Bros.,8.2,18th Oct 85,NES,Nintendo,Nintendo EAD,N/A
2,Mario Kart Wii,9.1,27th Apr 08,Wii,Nintendo,Nintendo EAD,8.7
3,PlayerUnknown's Battlegrounds,N/A,21st Dec 17,PC,PUBG Corporation,PUBG Corporation,N/A
4,Wii Sports Resort,8.8,26th Jul 09,Wii,Nintendo,Nintendo EAD,8.8
...,...,...,...,...,...,...,...
55859,ZOMBIE GOLD RUSH,N/A,26th Oct 17,NS,Amazing Inc.,Amazing Inc.,N/A
55860,Zombillie,N/A,29th Mar 18,NS,Forever Entertainment S.A.,Forever Entertainment S.A.,N/A
55861,Zone of the Enders: The 2nd Runner MARS,N/A,06th Sep 18,PC,Konami,Cygames,N/A
55862,Zoo Tycoon: Ultimate Animal Collection,N/A,31st Oct 17,XOne,Microsoft Studios,Frontier Developments,N/A


In [4]:
saleData = pd.read_csv("vgsales.csv")
saleData = saleData.drop(labels = ['Rank', 'Year', 'Publisher'], axis = 1)#Platform
saleData

,Name,Platform,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Wii Sports,Wii,Sports,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,Platform,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Wii,Racing,15.85,12.88,3.79,3.31,35.82
3,Wii Sports Resort,Wii,Sports,15.75,11.01,3.28,2.96,33.00
4,Pokemon Red/Pokemon Blue,GB,Role-Playing,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...
16593,Woody Woodpecker in Crazy Castle 5,GBA,Platform,0.01,0.00,0.00,0.00,0.01
16594,Men in Black II: Alien Escape,GC,Shooter,0.01,0.00,0.00,0.00,0.01
16595,SCORE International Baja 1000: The Official Game,PS2,Racing,0.00,0.00,0.00,0.00,0.01
16596,Know How 2,DS,Puzzle,0.00,0.01,0.00,0.00,0.01


In [5]:
finalData = saleData.merge(mainData, left_on=['Name', 'Platform'], right_on=['Name', 'console'])#, sort = True
finalData.drop(labels = ['console'], axis = 1, inplace = True)
finalData

,Name,Platform,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,user_Score,release_Date,publisher,developer,VGscore
0,Wii Sports,Wii,Sports,41.49,29.02,3.77,8.46,82.74,N/A,19th Nov 06,Nintendo,Nintendo EAD,N/A
1,Super Mario Bros.,NES,Platform,29.08,3.58,6.81,0.77,40.24,8.2,18th Oct 85,Nintendo,Nintendo EAD,N/A
2,Mario Kart Wii,Wii,Racing,15.85,12.88,3.79,3.31,35.82,9.1,27th Apr 08,Nintendo,Nintendo EAD,8.7
3,Wii Sports Resort,Wii,Sports,15.75,11.01,3.28,2.96,33.00,8.8,26th Jul 09,Nintendo,Nintendo EAD,8.8
4,Tetris,GB,Puzzle,23.20,2.26,4.22,0.58,30.26,N/A,01st Jun 89,Nintendo,Bullet Proof Software,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15890,Woody Woodpecker in Crazy Castle 5,GBA,Platform,0.01,0.00,0.00,0.00,0.01,N/A,02nd Dec 03,Kemco,Tantalus Interactive,N/A
15891,Men in Black II: Alien Escape,GC,Shooter,0.01,0.00,0.00,0.00,0.01,N/A,07th Feb 03,Infogrames,Melbourne House,N/A
15892,SCORE International Baja 1000: The Official Game,PS2,Racing,0.00,0.00,0.00,0.00,0.01,N/A,28th Oct 08,Activision,Left Field Productions,N/A
15893,Know How 2,DS,Puzzle,0.00,0.01,0.00,0.00,0.01,N/A,28th Oct 10,7G//AMES,Bitfield GmbH,N/A


In [6]:
finalData.sort_values(by = 'Global_Sales', ascending = False, axis = 0,ignore_index = True, inplace = True)
finalData['playformID'] = finalData.groupby('Platform').ngroup()
finalData['genreID'] = finalData.groupby('Genre').ngroup()
finalData['publisherID'] = finalData.groupby('publisher').ngroup()
finalData['developerID'] = finalData.groupby('developer').ngroup()
finalData['user_Score'] = pd.to_numeric(finalData['user_Score'], 'coerce')
finalData['VGscore'] = pd.to_numeric(finalData['VGscore'], 'coerce')
finalData

,Name,Platform,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,user_Score,release_Date,publisher,developer,VGscore,playformID,genreID,publisherID,developerID
0,Wii Sports,Wii,Sports,41.49,29.02,3.77,8.46,82.74,NaN,19th Nov 06,Nintendo,Nintendo EAD,NaN,24,10,353,1459
1,Super Mario Bros.,NES,Platform,29.08,3.58,6.81,0.77,40.24,8.2,18th Oct 85,Nintendo,Nintendo EAD,NaN,10,4,353,1459
2,Mario Kart Wii,Wii,Racing,15.85,12.88,3.79,3.31,35.82,9.1,27th Apr 08,Nintendo,Nintendo EAD,8.7,24,6,353,1459
3,Wii Sports Resort,Wii,Sports,15.75,11.01,3.28,2.96,33.00,8.8,26th Jul 09,Nintendo,Nintendo EAD,8.8,24,10,353,1459
4,Tetris,GB,Puzzle,23.20,2.26,4.22,0.58,30.26,NaN,01st Jun 89,Nintendo,Bullet Proof Software,NaN,5,5,353,357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15890,Satisfashion,Wii,Misc,0.01,0.00,0.00,0.00,0.01,NaN,08th Jun 10,Destineer,"eGames, Inc.",NaN,24,3,124,2386
15891,Homeworld Remastered Collection,PC,Strategy,0.00,0.01,0.00,0.00,0.01,NaN,25th Feb 15,Gearbox Software,Gearbox Software,NaN,12,11,188,851
15892,Metal Slug Anthology,PS2,Shooter,0.00,0.00,0.01,0.00,0.01,NaN,28th Mar 07,SNK Playmore,Terminal Reality,NaN,15,8,433,2093
15893,K-1 Grand Prix,PS,Fighting,0.01,0.01,0.00,0.00,0.01,NaN,25th Jan 00,Jaleco,Xing Entertainment,NaN,14,2,250,2336


In [7]:
cleansedData = finalData.drop(['Platform', 'Genre', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'publisher', 'developer'], axis = 1)
cleansedData['score'] = cleansedData[['VGscore', 'user_Score']].mean(axis = 1)
cleansedData = cleansedData.drop(['user_Score','VGscore'], axis = 1)
cleansedData['release_Date'] = pd.to_datetime(cleansedData['release_Date'], 'coerce', True, infer_datetime_format = True)
cleansedData.dropna(inplace = True, subset = ['release_Date'])

cleansedData

,Name,Global_Sales,release_Date,playformID,genreID,publisherID,developerID,score
0,Wii Sports,82.74,2006-11-19,24,10,353,1459,NaN
1,Super Mario Bros.,40.24,1985-10-18,10,4,353,1459,8.2
2,Mario Kart Wii,35.82,2008-04-27,24,6,353,1459,8.9
3,Wii Sports Resort,33.00,2009-07-26,24,10,353,1459,8.8
4,Tetris,30.26,1989-06-01,5,5,353,357,NaN
...,...,...,...,...,...,...,...,...
15890,Satisfashion,0.01,2010-06-08,24,3,124,2386,NaN
15891,Homeworld Remastered Collection,0.01,2015-02-25,12,11,188,851,NaN
15892,Metal Slug Anthology,0.01,2007-03-28,15,8,433,2093,NaN
15893,K-1 Grand Prix,0.01,2000-01-25,14,2,250,2336,NaN


In [31]:
testdata = cleansedData.copy()

#testdata.iloc[0:int(len(testdata)*.25), 1] = 'High'
#testdata.iloc[int(len(testdata)*.25) + 1:int(len(testdata)*.50), 1] = 'Medium'
#testdata.iloc[int(len(testdata)*.50) + 1:int(len(testdata)*.75), 1] = 'Low'
#testdata.iloc[int(len(testdata)*.75) + 1:, 1] = 'Abysmal'

testdata

,Name,Global_Sales,release_Date,playformID,genreID,publisherID,developerID,score
0,Wii Sports,82.74,2006-11-19,24,10,353,1459,NaN
1,Super Mario Bros.,40.24,1985-10-18,10,4,353,1459,8.2
2,Mario Kart Wii,35.82,2008-04-27,24,6,353,1459,8.9
3,Wii Sports Resort,33.00,2009-07-26,24,10,353,1459,8.8
4,Tetris,30.26,1989-06-01,5,5,353,357,NaN
...,...,...,...,...,...,...,...,...
15890,Satisfashion,0.01,2010-06-08,24,3,124,2386,NaN
15891,Homeworld Remastered Collection,0.01,2015-02-25,12,11,188,851,NaN
15892,Metal Slug Anthology,0.01,2007-03-28,15,8,433,2093,NaN
15893,K-1 Grand Prix,0.01,2000-01-25,14,2,250,2336,NaN


In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [36]:
finalizingData = testdata.drop(labels = ["Name", "release_Date", "score"], axis = 1)
print(finalizingData)
#xtrain, ytrain, xtest, ytest = train_test_split(, axis = 1), testdata["Global_Sales"], test_size = .25)



#clf = RandomForestClassifier(max_depth=2, random_state=0)
#clf.fit(xtrain, ytrain)

       Global_Sales release_Date  playformID  genreID  publisherID  \
0             82.74   2006-11-19          24       10          353   
1             40.24   1985-10-18          10        4          353   
2             35.82   2008-04-27          24        6          353   
3             33.00   2009-07-26          24       10          353   
4             30.26   1989-06-01           5        5          353   
...             ...          ...         ...      ...          ...   
15890          0.01   2010-06-08          24        3          124   
15891          0.01   2015-02-25          12       11          188   
15892          0.01   2007-03-28          15        8          433   
15893          0.01   2000-01-25          14        2          250   
15894          0.01   2003-11-02           6        4          132   

       developerID  score  
0             1459    NaN  
1             1459    8.2  
2             1459    8.9  
3             1459    8.8  
4              357 

In [8]:
#notes- 
#Using name might be a bit too baised to series, but still could work
#Everything should be able to be ID'd in some way, forgot the name of it but we did it with the text message thing for spam/ham
#That'd be good for names if we were to do it, each unique genre/console/publisher/developer should be given a wholly unique
#single ID.
#Scores should be averaged together where available to reduce columns by one, perhaps we shoot for the total sales as well and 
#drop inidivudal categories of sales.

#Date could be months since the earliest entry or just like 1980
#Could eventually use the day to nudge/round the month or even check which day of the week it released on, like sunday/monday/
#friday

#Julian timestamps